# Use RAPIDS on multiple GPUs in parallel with Dask

We perform the same machine learning exercise as the previous notebook, except on a cluster of multiple GPUs with Dask

## Use RAPIDS libraries

RAPIDS is a collection of libraries which enable you to take advantage of NVIDIA GPUs to accelerate machine learning workflows. This exercise uses the following RAPIDS packages:
    
* [`dask-cudf`](https://docs.rapids.ai/api/cudf/stable/dask-cudf.html): distributed `cudf` dataframes using Dask
* [`cuml`](https://github.com/rapidsai/cuml): machine learning training and evaluation, similar to `scikit-learn`

<hr>

## Initialize A Dask Cluster

This project has a Dask cluster defined for it, which you can start or connect to in the below cell. For more information about Dask clusters in Saturn Cloud, check out [the docs](https://www.saturncloud.io/docs/examples/dask/).

In [ ]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster()
client = Client(cluster)
client.wait_for_workers(n_workers=n_workers)

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of multiple GPUs for data science workflows. To prove to yourself that Dask RAPIDS are utilizing the GPUs, it's important to understand how to monitor that utilization while your code is running. If you already know how to do that, skip to the next section.

Print the `cluster` object in a notebook renders a widget that shows the number of workers, available CPU and memory, and a dashboard link.

In [ ]:
cluster

Click that dashboard link to view some diagnostic information about the Dask cluster. This can be used to view the current resource utilization of workers in the cluster and lots of information about what they're currently working on.

There are a couple other dashboard pages worth viewing for GPU memory and utilization that are not listed on the navbar. Run the code below to view them.

In [ ]:
from IPython.display import display, HTML

gpu_links = f"""
<b>GPU Dashboard links</b>
<ul>
<li><a href="{client.dashboard_link}/individual-gpu-memory" target="_blank">GPU memory</a></li>
<li><a href="{client.dashboard_link}/individual-gpu-utilization" target="_blank">GPU utilization</a></li>
</ul>
"""
display(HTML(gpu_links))

<hr>

## Load data

This example is designed to run quickly with small resources. So let's just load a single month of taxi data for training.

The code below loads the data into a `dask-cudf` data frame. Your code here in this notebook can interact with this data structure as if it was just a regular `cudf` data frame, but it is actually a collection of smaller `cudf` data frames spread across the workers in the Dask cluster.

A `cudf` data frame is similar to a `pandas` data frame, but it lives in GPU memory and most operations on it are done on the GPU.

In [ ]:
import dask_cudf

taxi = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

The code below computes the size of this dataset in memory. One month is about 7.6 million rows and 1.08 GB.

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e9} GB")

You can examine the structure of the data with `cudf` commands:

`.head()` = view the first few rows

In [ ]:
taxi.head()

`.dtypes` = list all the columns and the type of data in them

In [ ]:
taxi.dtypes

When we say that a `dask-cudf` dataframe is a *distributed* data frame, that means that it comprises multiple smaller `cudf` data frames. Run the following to see how many of these pieces (called "partitions") there are.

In [ ]:
taxi.npartitions

<hr>

## Prep for Training

Notice that this code looks exactly the same as the single-GPU RAPIDS version

In [ ]:
def prep_df(df: dask_cudf.DataFrame, target_col: str) -> dask_cudf.DataFrame:
    """
    Prepare a raw taxi dataframe for training.
        * computes the target ('high_tip')
        * adds features
        * removes unused features

    Casts all numeric values to 32-bit types, for efficiency and
    because some older versions of CUDA / ``cudf``, ``cuml``
    did not support 64-bit types in training data.
    """
    numeric_feat = [
        "pickup_weekday",
        "pickup_hour",
        "pickup_week_hour",
        "pickup_minute",
        "passenger_count",
    ]
    categorical_feat = [
        "PULocationID",
        "DOLocationID",
    ]
    features = numeric_feat + categorical_feat

    # add target
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df["tip_fraction"] = df.tip_amount / df.fare_amount
    df[target_col] = df["tip_fraction"] > 0.2

    # add features
    df["pickup_weekday"] = df.tpep_pickup_datetime.dt.weekday
    df["pickup_hour"] = df.tpep_pickup_datetime.dt.hour
    df["pickup_week_hour"] = (df.pickup_weekday * 24) + df.pickup_hour
    df["pickup_minute"] = df.tpep_pickup_datetime.dt.minute

    # drop unused columns
    df = df[features + [target_col]].astype("float32").fillna(-1)

    # convert target to int32 for efficiency (it's just 0s and 1s)
    df[target_col] = df[target_col].astype("int32")

    return df

Run the code below to get a new data frame, `taxi_train`, that can be used directly for model training.

In [ ]:
target_col = "high_tip"

taxi_train = prep_df(df=taxi, target_col=target_col)

Dask performs computations in a [lazy manner](https://tutorial.dask.org/01x_lazy.html), so we persist the dataframe to perform data loading and feature processing and load into GPU memory.

In [ ]:
%%time
taxi_train = taxi_train.persist()
_ = wait(taxi_train)

In [ ]:
taxi_train.groupby("high_tip")["high_tip"].count().compute()

Before going further, check the first few rows of the dataset to make sure that the features look reasonable.

In [ ]:
taxi_train.head()

Now that the dataframe has been processed, check its size in memory again.

In [ ]:
print(
    f"Num rows: {len(taxi_train)}, Size: {taxi_train.memory_usage(deep=True).sum().compute() / 1e9} GB"
)

As you can see above, removing unused columns dropped the size of the training data to 0.31 GB, about one third the size of the raw data.

<hr>

## Train a Model

Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `RandomForestClassifier` from `cuml`. If you've never used a random forest or need a refresher, consult ["Forests of randomized trees"](https://scikit-learn.org/stable/modules/ensemble.html#forest) in the `sciki-learn` documentation.

In [ ]:
from cuml.dask.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10)

In [ ]:
%%time

features = [c for c in taxi_train.columns if c != target_col]

rfc.fit(taxi_train[features], taxi_train[target_col])

<hr>

## Calculate metrics on test set

We'll use another month of taxi data for the test set

In [ ]:
taxi_test = dask_cudf.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    assume_missing=True,
)

Before creating predictions on this new dataset, it has to be transformed in exactly the way that the original training data were prepared. Thankfully you've already wrapped that transformation logic in a function!

In [ ]:
taxi_test = prep_df(taxi_test, target_col=target_col)

In [ ]:
from cuml.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[1]

In [ ]:
roc_auc_score(taxi_test[target_col].compute(), preds.compute())